# **PROTEINS AND DRUG DESIGN**
### Bojos per la Supercomputació | 13 Abril

- Authors: Isaac Filella, Júlia Vilalta
- Contact: isaac.filella1@bsc.es, julia.vilalta@bsc.es

# 0.1. Molecular Docking

Molecular docking simulations are computational techniques that play a crucial role in drug design by emulating interactions between small molecules (ligands) and macromolecular targets (proteins). These methods computationally predict the binding mode (pose) and binding affinity of ligands within the binding sites of target proteins, thereby providing critical insights into the molecular mechanisms underlying ligand-receptor interactions.

<figure>
<center>
<img src='https://raw.githubusercontent.com/IFilella/bojos_supercomputacio/main/imatges/docking.png'/>
</figure>

Molecular docking programs employ a search algorithm that iteratively assesses different ligand poses until converging to an energy minimum. Then, using a scoring function, they estimate the binding free energy in kcal/mol to rank candidate poses.

# 0.2. Biological context

KRAS (Kirsten rat sarcoma viral oncogene homolog) is a critical gene encoding a small GTPase protein involved in cell signaling pathways regulating cell growth and proliferation. Mutations in KRAS have been associated with fatal cancers, including pancreatic, lung, and colorectal cancers, making it a target protein for cancer research and drug design efforts. One of these oncogenic mutations is KRAS<sup>G12D</sup>. Recent studies have shown that an allosteric pocket (binding site) named SII can be used to inhibit KRAS<sup>G12D</sup> malfunction and, hence, stop cancer proliferation. Mirati's (pharmaceutical company) research team particularly discovered MRTX1133, an SII ligand that induces KRAS pathway inhibition.

<figure>
<center>
<img src='https://raw.githubusercontent.com/IFilella/bojos_supercomputacio/main/imatges/KRASG12D_MRTX1133.png'/>
<figcaption>Experimental known pose of MRTX1133 while inhibiting KRAS<sup>G12D</sup> protein </i></figcaption></center>
</figure>

# 0.2. Practical Session

We will start this practical session with a test in which we will dock a known ligand inhibitor to KRAS<sup>G12D</sup>. With this initial test, we aim to evaluate how well a computational docking method reproduces the **true** ligand's experimentally known pose within the target protein's binding site. Secondly, we will focus on identifying new ligands capable of inhibiting KRAS<sup>G12D</sup> by conducting a virtual screening (VS) using docking procedures. Finally, we rank the newly identified ligands compared to the known ligand inhibitor used in the initial test.

<figure>
<center>
<img src='https://raw.githubusercontent.com/IFilella/bojos_supercomputacio/main/imatges/docking_workflow.png'/>
<figcaption>Molecular docking general steps. First, the target protein and the ligand are prepared. Then, the system (ligand + target protein) is prepared by setting up the search grid (region where the exploration is going to be performed). After the docking, the final ligand poses are scored based on a scoring function. Finally, the results are compared against experimental data for validation.</i></figcaption></center>
</figure>

# 1. Downloading and Installing the required programs

We should start by  installing several software (this could take a few minuts):


In [1]:
%%capture
#Installing py3Dmol using pip
!pip -q install py3Dmol
#Installing biopython using pip
!pip -q install biopython
#Install conda using conda-colab library
!pip install -q condacolab
#Installing pdb2pqr v3.0 using pip
!pip -q install pdb2pqr
#Installing RDkit using pip
!pip -q install rdkit-pypi
#
!pip install meeko
!pip install scipy
!pip install pandas

In [2]:
%%capture
import condacolab
condacolab.install_miniconda()
#Install MGLtools and OpenBabel from the bioconda repository
!conda install -c conda-forge -c bioconda mgltools openbabel zlib ncurses --yes

In [1]:
#Download and extract Autodock Vina from SCRIPPS
%%bash
mkdir vina
cd vina
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
chmod +x /content/vina/vina_1.2.5_linux_x86_64
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_split_1.2.5_linux_x86_64
chmod +x /content/vina/vina_split_1.2.5_linux_x86_64

In [2]:
#Set up an alias for vina to be treated as a native binary
%alias vina /content/vina/vina_1.2.5_linux_x86_64
%alias vina_split /content/vina/vina_1.2.5_linux_x86_64

# 2. Test: MRTX1133 ligand docking into KRAS<sup>G12D</sup> target protein

First we will upload the file cotaining the target protein (KRAS<sup>G12D</sup>) with the ligand (MRTX1133) on its **true** experimentally known pose:

In [ ]:
#@title 2.1 Uploading experimental structure
import os
from pathlib import Path
from google.colab import files

experimental_path = Path("/content/experimental/")

if os.path.exists(experimental_path):
  print("exp. path already exists")
if not os.path.exists(experimental_path):
  os.mkdir(experimental_path)
  print("exp. path was succesfully created")

fl = files.upload();

!mv krasg12d_mrtx1133_experimental.pdb experimental/

exp_protein_file = 'experimental/krasg12d_mrtx1133_experimental.pdb'

Second, we will visualize KRAS<sup>G12D</sup> with MRTX1133 on its true experimental known pose:

In [4]:
#@title 2.2 Visualization experimental
import py3Dmol
view=py3Dmol.view()
view.addModel(open(exp_protein_file, 'r').read(),'pdb')
view.zoomTo()
#Setting the background color as white
view.setBackgroundColor('lightgray')
#Settinh the visualization style for chain A
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})
view.addSurface(py3Dmol.SES, {'opacity':0.7, 'color':'grey'}, \
  {'not':{'or':[{'resn':'6IC'}, {'resn':'GDP'}]}})
#Hide ligands
view.setStyle({'chain':'L'},{'hidden': True})
#Add a visualization style for ligand
view.addStyle({'chain':'L','resi':202},{'stick':{'colorscheme':'green'}})

view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### 2.3 Preparing the protein target for docking

To prepare the protein target for docking, we will start by removing the MRTX1133 ligand as well as any other molecule that could be in the file (e.g., water molecules, ions, ...). This can be quickly done by extracting all of the lines from the PDB file that start with "ATOM", as this is how all of the atoms that belong to the protein are termed. In contrast, the atoms from ligands, water molecules, and other non-protein molecules are commonly referred to as "HETATM".

In [5]:
with open(experimental_path / "krasg12d_prep.pdb","w") as g:
  f = open(exp_protein_file,'r')
  for line in f:
    row = line.split()
    if row[0] == "ATOM":
      g.write(line)
    elif row[0] == "TER":
      g.write("TER\n")
  g.write("END")
  print("file successfully created")

file successfully created



Next, in preparation for the specific docking program, AutoDock Vina, in our case, we must generate a PDBQT file of the target protein. This requires executing the following commands that will also incorporate hydrogen atoms into the protein structure.

In [6]:
%%capture
!/usr/local/bin/python2 /usr/local/bin/prepare_receptor4.py -r $experimental_path/krasg12d_prep.pdb -o $experimental_path/krasg12d_prep.pdbqt -A hydrogens -U nphs_lps -v

In [7]:
#@title Visualization of prepared target protein
import py3Dmol
view=py3Dmol.view()
view.addModel(open('/content/experimental/krasg12d_prep.pdbqt', 'r').read(),'pdb')
view.zoomTo()
#Setting the background color as white
view.setBackgroundColor('lightgray')
#Settinh the visualization style for chain A
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})
view.addSurface(py3Dmol.SES, {'opacity':0.7, 'color':'grey'}, \
  {'not':{'or':[{'resn':'6IC'}, {'resn':'GDP'}]}})
#view.addSphere({center:{x:0,y:0,z:0},radius:10.0,color:'red'});


view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### 2.4 Preparing the ligand MRTX1133 for docking

extracting the ligand

In [8]:
with open(experimental_path / "mrtx1133_prep.pdb","w") as g:
  f = open(exp_protein_file,'r')
  for line in f:
    row = line.split()
    if row[0] == "HETATM" and row[5] == '202':
      g.write(line)
  g.write("END")
  print("file successfully created")

file successfully created


**preparing** the ligand (it can take some minutes)

In [9]:
!obabel $experimental_path/mrtx1133_prep.pdb -O mrtx1133_prep.mol2 --gen3d best -p 7.4 --canonical
!/usr/local/bin/python2 /usr/local/bin/prepare_ligand4.py -l mrtx1133_prep.mol2 -o $experimental_path/mrtx1133_prep.pdbqt -U nphs_lps -v
os.system('mv mrtx1133_prep.mol2 experimental/')

*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 2 stereocenter(s) in this molecule (/content/experimental/mrtx1133_prep.pdb)
  with Atom Ids as follows: 20 24
*** Open Babel Warning  in Build
  There exists NaN in calculated coordinates.
1 molecule converted
set verbose to  True
read  mrtx1133_prep.mol2
setting up LPO with mode= automatic and outputfilename=  /content/experimental/mrtx1133_prep.pdbqt
and check_for_fragments= False
and bonds_to_inactivate= 
returning  0
No change in atomic coordinates


0

In [10]:
#@title Visualizing the ligand
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

ligand_file = '/content/experimental/mrtx1133_prep.pdb'
molecule = Chem.MolFromPDBFile(ligand_file)
#molecule = mol_supplier[9]

AllChem.EmbedMolecule(molecule)
AllChem.MMFFOptimizeMolecule(molecule)
mol_block = Chem.MolToMolBlock(molecule)

viewer = py3Dmol.view(width=400, height=400)
viewer.addModel(mol_block, "mol")
viewer.setStyle({'stick': {}})
viewer.setBackgroundColor('lightgray')
viewer.zoomTo()
viewer.show()


[10:54:06] Molecule does not have explicit Hs. Consider calling AddHs()
[10:54:07] Molecule does not have explicit Hs. Consider calling AddHs()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### 2.5 Docking

In [19]:
#@title Generating an AutoDock Vina file
receptor = "experimental/krasg12d_prep.pdbqt" # @param {type:"string"}
ligand = "experimental/mrtx1133_prep.pdbqt" # @param {type:"string"}
center_x = "-0.362" # @param {type:"string"}
center_y = "3.360" # @param {type:"string"}
center_z = "-24.427" # @param {type:"string"}
size_x = "20" # @param {type:"string"}
size_y = "20" # @param {type:"string"}
size_z = "20" # @param {type:"string"}
with open("config_experimental","w") as f:
  f.write("#CONFIGURATION FILE (options not used are commented) \n")
  f.write("\n")
  f.write("#INPUT OPTIONS \n")
  f.write("receptor = " + receptor +"\n")
  f.write("ligand = " + ligand +"\n")
  f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
  f.write("#SEARCH SPACE CONFIGURATIONS \n")
  f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES
  f.write("center_x = " + center_x + "\n")
  f.write("center_y = " + center_y + "\n")
  f.write("center_z = " + center_z + "\n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
  f.write("#Size of the box (values bxf, byf and bzf) \n")
  f.write("size_x = " + size_x + "\n")
  f.write("size_y = " + size_y + "\n")
  f.write("size_z = " + size_z + "\n")
  f.write("#OUTPUT OPTIONS \n")
  f.write("#out = experimental/ \n")
  f.write("#log = experimental/ \n")
  f.write("\n")
  f.write("#OTHER OPTIONS \n")
  f.write("#cpu =  \n")
  f.write("#exhaustiveness = \n")
  f.write("#num_modes = \n")
  f.write("#energy_range = \n")
  f.write("#seed = ")

docking with autodock vina (can take a while)

In [20]:
%vina --config config_experimental --out output.pdbqt | tee output.log

AutoDock Vina v1.2.5
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [ ]:
!mv output.log experimental/
!mv output.pdbqt experimental/
!obabel experimental/output.pdbqt -O experimental/output.pdb

2 molecules converted


In [ ]:
#@title 2.6 Visualization of the results
import py3Dmol
view=py3Dmol.view()
view.addModel(open(exp_protein_file, 'r').read(),'pdb')
view.zoomTo()
#Setting the background color as white
view.setBackgroundColor('lightgray')
#Settinh the visualization style for chain A
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})
#view.addSurface(py3Dmol.VDW, {'opacity':0.7, 'color':'grey'}, \
#  {'not':{'or':[{'resn':'6IC'}, {'resn':'GDP'}]}})
#Hide ligands
view.setStyle({'chain':'L'},{'hidden': True})
#Add a visualization style for ligand
view.addStyle({'chain':'L','resi':202},{'stick':{'colorscheme':'lightgreen'}})

view.addModel(open('experimental/output.pdb', 'r').read(),'pdb')
view.setStyle({'model':1}, {'stick': {'color':'orange'}})
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 3. Virtual screening against KRAS<sup>G12D</sup>

We start by uploading the file cotaining the new ligands that we want to dock and rank against (KRAS<sup>G12D</sup>):

In [21]:
#@title 3.1 Uploading new ligands

import os
from pathlib import Path
from google.colab import files

vs_path = Path("/content/virtual_screening/")

if os.path.exists(vs_path):
  print("vh path already exists")
if not os.path.exists(vs_path):
  os.mkdir(vs_path)
  print("vh path was succesfully created")

fl = files.upload();

!mv ligands.sdf $vs_path

ligands = '/content/virtual_screening/ligands.sdf'

vh path was succesfully created


Saving ligands.sdf to ligands.sdf


In [23]:
#@title 3.2 Visualizing the compounds (0 to 7)
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

ligands = '/content/virtual_screening/ligands.sdf'
molecules = Chem.SDMolSupplier(ligands)
molecule = 4 # @param {type:"integer"}
mol = molecules[molecule]

AllChem.EmbedMolecule(mol)
AllChem.MMFFOptimizeMolecule(mol)
mol_block = Chem.MolToMolBlock(mol)

viewer = py3Dmol.view(width=400, height=400)
viewer.addModel(mol_block, "mol")
viewer.setStyle({'stick': {}})
viewer.setBackgroundColor('lightgray')
viewer.zoomTo()
viewer.show()

[11:16:37] Molecule does not have explicit Hs. Consider calling AddHs()
[11:16:37] Molecule does not have explicit Hs. Consider calling AddHs()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### 3.3 Preparing the ligands

In [24]:
!mk_prepare_ligand.py -i virtual_screening/ligands.sdf --multimol_outdir virtual_screening/ --multimol_prefix ligand

Input molecules processed: 8, skipped: 0
PDBQT files written: 8
PDBQT files not written due to error: 0
Input molecules with errors: 0
No duplicate molecule molecule names were found


In [25]:
#@title 3.4 Generating an AutoDock Vina file
receptor = "experimental/krasg12d_prep.pdbqt" # @param {type:"string"}
center_x = "-0.362" # @param {type:"string"}
center_y = "3.360" # @param {type:"string"}
center_z = "-24.427" # @param {type:"string"}
size_x = "20" # @param {type:"string"}
size_y = "20" # @param {type:"string"}
size_z = "20" # @param {type:"string"}
with open("config_vs","w") as f:
  f.write("#CONFIGURATION FILE (options not used are commented) \n")
  f.write("\n")
  f.write("#INPUT OPTIONS \n")
  f.write("receptor = " + receptor +"\n")
  f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
  f.write("#SEARCH SPACE CONFIGURATIONS \n")
  f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES
  f.write("center_x = " + center_x + "\n")
  f.write("center_y = " + center_y + "\n")
  f.write("center_z = " + center_z + "\n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
  f.write("#Size of the box (values bxf, byf and bzf) \n")
  f.write("size_x = " + size_x + "\n")
  f.write("size_y = " + size_y + "\n")
  f.write("size_z = " + size_z + "\n")
  f.write("#OUTPUT OPTIONS \n")
  f.write("#out = experimental/ \n")
  f.write("#log = experimental/ \n")
  f.write("\n")
  f.write("#OTHER OPTIONS \n")
  f.write("#cpu =  \n")
  f.write("#exhaustiveness = \n")
  f.write("#num_modes = \n")
  f.write("#energy_range = \n")
  f.write("#seed = ")

### 3.5 Executing AutoDock Vina

In [26]:
%vina --config config_vs --ligand virtual_screening/ligand-1.pdbqt --out output1.pdbqt | tee output1.log
%vina --config config_vs --ligand virtual_screening/ligand-2.pdbqt --out output2.pdbqt | tee output2.log
%vina --config config_vs --ligand virtual_screening/ligand-3.pdbqt --out output3.pdbqt | tee output3.log
%vina --config config_vs --ligand virtual_screening/ligand-4.pdbqt --out output4.pdbqt | tee output4.log
%vina --config config_vs --ligand virtual_screening/ligand-5.pdbqt --out output5.pdbqt | tee output5.log
%vina --config config_vs --ligand virtual_screening/ligand-6.pdbqt --out output6.pdbqt | tee output6.log
%vina --config config_vs --ligand virtual_screening/ligand-7.pdbqt --out output7.pdbqt | tee output7.log
%vina --config config_vs --ligand virtual_screening/ligand-8.pdbqt --out output8.pdbqt | tee output8.log

AutoDock Vina v1.2.5
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [28]:
!mv output*.log virtual_screening/
!mv output*.pdbqt virtual_screening/
!obabel virtual_screening/output1.pdbqt -O virtual_screening/output1.pdb
!obabel virtual_screening/output2.pdbqt -O virtual_screening/output2.pdb
!obabel virtual_screening/output3.pdbqt -O virtual_screening/output3.pdb
!obabel virtual_screening/output4.pdbqt -O virtual_screening/output4.pdb
!obabel virtual_screening/output5.pdbqt -O virtual_screening/output5.pdb
!obabel virtual_screening/output6.pdbqt -O virtual_screening/output6.pdb
!obabel virtual_screening/output7.pdbqt -O virtual_screening/output7.pdb
!obabel virtual_screening/output8.pdbqt -O virtual_screening/output8.pdb

9 molecules converted
9 molecules converted
9 molecules converted
9 molecules converted
9 molecules converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is virtual_screening/output6.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is virtual_screening/output6.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is virtual_screening/output6.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is virtual_screening/output6.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is virtual_screening/output6.pdbqt)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (tit

In [29]:
for i in range(8):
  fin = open('virtual_screening/output%d.pdb'%(i+1), 'r')
  count = 0
  fout = open('virtual_screening/output%d_%d.pdb'%(i+1,count),'w')
  for line in fin:
    if 'ENDMDL\n' in line:
      fout.close()
      count+=1
      fout = open('virtual_screening/output%d_%d.pdb'%(i+1,count),'w')
      continue
    fout.write(line)

In [30]:
#@title 3.6 Visualization of the results
import py3Dmol
view=py3Dmol.view()
view.addModel(open('experimental/krasg12d_prep.pdb', 'r').read(),'pdb')
view.zoomTo()
#Setting the background color as white
view.setBackgroundColor('lightgray')
#Settinh the visualization style for chain A
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})
#view.addSurface(py3Dmol.VDW, {'opacity':0.7, 'color':'grey'}, \
#  {'not':{'or':[{'resn':'6IC'}, {'resn':'GDP'}]}})
#Hide ligands
#view.setStyle({'chain':'L'},{'hidden': True})
#Add a visualization style for ligand
#view.addStyle({'chain':'L','resi':202},{'stick':{'colorscheme':'lightgreen'}})

molecule = 4 # @param {type:"integer"}
view.addModel(open('virtual_screening/output%d_1.pdb'%molecule, 'r').read(),'pdb')
view.setStyle({'model':1}, {'stick':{'colorscheme':'green'}})
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.